In [75]:
from openai import OpenAI
from dotenv import load_dotenv
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import time
import requests

load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

pd.set_option('display.max_rows', 100)

In [81]:
df = pd.read_csv('data/fetchGPT.csv')

# Get 50 entries each from Costco and Whole Foods Market
costco_df = df[df['SAMPLE_STORE'] == 'COSTCO'].head(500)
wholefoods_df = df[df['SAMPLE_STORE'] == 'WHOLE FOODS MARKET'].head(500)

# Combine the two dataframes
temp = pd.concat([costco_df, wholefoods_df], ignore_index=True)

# Verify the distribution
# print("Store distribution:")
# print(temp['SAMPLE_STORE'].value_counts())
temp

,ORIGINAL_ITEM_TEXT,FIDO_TYPE,DESCRIPTION,TIER1_NAME,BRAND,FIDO,PRODUCT_NUMBER,SAMPLE_STORE,SAMPLE_RECEIPT,ITEM_COUNT,MEDIAN(PAM.FINAL_PRICE),GMV
0,strawberries,PRODUCT,Strawberries,Produce,UNBRANDED,d18956ad-4eaa-408d-9620-26256eed1e4f,27003,COSTCO,6984257f-ae79-417b-9407-9d7fa9135954,348691,4.49,1565622.59
1,***kswtr40pk,PRODUCT,Kirkland Signature Purified Drinking Water,Beverages,KIRKLAND SIGNATURE,97f8b18c-640d-4105-9a2f-97f726996c5b,782796,COSTCO,14094048-0423-4e8e-8fa1-27f78b5823f3,283476,3.99,1131069.24
2,rotisserie,PRODUCT,Kirkland Signature Rotisserie Chicken,Deli & Bakery,KIRKLAND SIGNATURE,175c65d7-e9ce-4108-838c-8148098892ec,87745,COSTCO,dd890074-a2d9-4aaa-9f1b-9e5c48717e01,258676,4.99,1290793.24
3,ks cage free,PRODUCT,Kirkland Signature Large Eggs-cage Free-2 Dozen,Dairy,KIRKLAND SIGNATURE,12a677be-8339-42cc-9c15-495d6a80c0c0,637598,COSTCO,e2053422-40ba-4bb4-afc0-be69182020f2,189662,6.89,1306771.18
4,18 ct eggs,PRODUCT,Sauders Eggs Large White 18 Ct,Dairy,SAUDER,5875bf31-3674-4ceb-a4cc-437bc46ec250,1008,COSTCO,17d39fed-4a3a-463b-82b6-eac9df1823d1,162319,5.19,842435.61
...,...,...,...,...,...,...,...,...,...,...,...,...
995,gv pink salt,PRODUCT,Great Value Fine Himalayan Pink Salt - 17.6 Oz - Pack Size Single,Pantry,GREAT VALUE,78104f3e-6c0c-454e-ad14-364a67ffef73,078742368740,WHOLE FOODS MARKET,c88e12a4-1029-429e-a563-93cd057594ab,2,5.88,11.76
996,fncy dlght turk chpc,PRODUCT,Fancy Feast Grilled Turkey And Cheddar Cheese Feast In Gravy - 3 Oz,Animals & Pet Supplies,FANCY FEAST,c3b610b7-8bef-41db-b8fe-1df7713b43ba,NaN,WHOLE FOODS MARKET,a864a0d6-43e1-4f11-b3a8-5aa9915b8afe,2,0.95,1.90
997,yellow onions,PRODUCT,YELLOW ONIONS,Produce,UNBRANDED,6c71a46b-d8a1-431c-acf5-35c69d3bdd3b,NaN,WHOLE FOODS MARKET,c35ed3db-f236-44d8-bc54-ffcf7fd7399c,2,5.84,11.68
998,bonipak iceberg lettuce 1,PRODUCT,Iceberg Lettuce,Produce,UNBRANDED,8c1a19f0-18d3-4981-9c86-75291c7fc0b5,0003338365020,WHOLE FOODS MARKET,97799793-5e24-4932-81fc-82f23b9a3b94,2,2.99,5.98


In [82]:

# --- Assumptions: ---
# 1. `temp` is your pandas DataFrame with columns:
#      - 'ORIGINAL_ITEM_TEXT'
#      - 'SAMPLE_STORE'
#      - optionally 'PRODUCT_NUMBER'
# 2. `client` is already initialized and authenticated, e.g.:
#      client = OpenAI(api_key="YOUR_KEY")

results = []

for idx, row in temp.iterrows():
    original_item_text = row['ORIGINAL_ITEM_TEXT']
    sample_store      = row['SAMPLE_STORE']
    product_number    = row.get('PRODUCT_NUMBER', 'N/A')
    median_price      = row.get('MEDIAN(PAM.FINAL_PRICE)', 'N/A')
    
    prompt = f'''
    # Role and Objective
    You are an AI product description analyzer tasked with standardizing and expanding abbreviated product descriptions into clear, structured data. Your goal is to identify brands, categories, and expand abbreviated text while maintaining accuracy.

    # Instructions
    - Analyze the given abbreviated product description
    - Expand abbreviations without adding interpretive content
    - Identify the most likely brand based on text and product number
    - Categorize the product based on expanded description
    - Provide confidence scores for brand and category predictions

    ## Sub-categories for more detailed instructions
    - Expand what is directly implied in the text (e.g., "gl" to "glass", "bl" to "bottle"), unless there are specific annotations like counts or sizes that can be verified through web search, or brand.
    - Assign confidence scores from high/medium/low based on clarity of information
    - Consider store context when determining brand and category

    # Reasoning Steps
    1. Expand Abbreviations
       - Identify common product abbreviations
       - Convert to standard product terminology
       - Maintain original meaning without interpretation
       - The format should be: brand name, product type, size, packaging, provided that this information is available through web search. 

    2. Brand Analysis
       - Look for brand indicators in text
       - Consider store-specific context
       - Assess confidence in brand identification

    3. Category Assignment
       - Analyze product characteristics
       - Determine product type
       - Assign to the closest predefined category in the available category taxonomy
       - Assign confidence based on clarity

       Available Category Taxonomy:
        - Savings & Coupons
        - Electronics
        - Home & Garden
        - Pantry
        - Beverages
        - Office & School
        - Household Supplies
        - Health & Wellness
        - Apparel & Accessories
        - Meat & Seafood
        - Frozen
        - Baby & Toddler
        - Vehicles & Parts
        - Beauty
        - Deli & Bakery
        - Scientific & Laboratory
        - Animals & Pet Supplies
        - Restaurant
        - Membership and Perks
        - Alcohol
        - Sporting Goods
        - Snacks
        - Commodities
        - Luggage & Bags
        - Media
        - Retail
        - Arts & Entertainment
        - Fees
        - Dairy
        - Toys & Games
        - Mature
        - Software
        - Produce


    # Output Format
    JSON structure with:
    {{
        "brand": "Predicted brand",
        "brand_score": "Confidence score (high/medium/low)",
        "category": "Predicted category",
        "category_score": "Confidence score (high/medium/low)",
        "expanded_description": "Expanded product description",
        "reasoning": "Reasoning for predictions and description"
    }}

    # Examples
    ## Example 1
    Input: "campari 12oz gl bl"
    Output: {{
        "brand": "Campari",
        "brand_score": high,
        "category": "Alcohol",
        "category_score": high,
        "expanded_description": "Campari 12oz Glass Bottle",
        "reasoning": "Clear brand name 'Campari' present. Common abbreviations 'gl bl' clearly indicate glass bottle. Spirit category evident from brand."
    }}

    ## Example 2
    Input: "18 ct eggs"
    Output: {{
        "brand": "Sauders",
        "brand_score": low,
        "category": "Dairy",
        "category_score": high,
        "expanded_description": "Sauders Eggs Large White 18 Ct	",
        "reasoning": "Product number 1008 at retailer COSTCO leads to Sauders Large Eggs."
    }}

    # Context
    Here is the product information:
    - Original Item Text: "{original_item_text}"
    - Store: "{sample_store}"
    - Product Number: "{product_number}" (if applicable)
    - Median Price: "${median_price}" (if applicable)
    
    # Additional Considerations
    - Use price as a signal for brand tier (premium/standard/value)
    - Consider price ranges typical for the category
    - Factor price into confidence scores when relevant

    # Final instructions and prompt to think step by step
    1. First, expand only the abbreviated terms in the original text
    2. Then identify brand based on expanded text
    3. Finally, categorize the product based on the complete information
    4. Provide clear reasoning for each decision
    5. Return structured JSON with confidence scores
    '''
    
    # Common Abbreviations and Terms
    # [Reserved for future implementation]

    # Store-Specific Categories
    # [Reserved for future implementation]

    # Product Naming Conventions
    # [Reserved for future implementation]


    # Send the prompt to the GPT model
    response = client.responses.create(
        model="gpt-4.1",
        tools=[{"type": "web_search_preview"}],
        input=prompt, 
        temperature=0.0
    )
    
    # Extract the JSON payload from the response
    output_text = response.output_text
    start = output_text.find('{')
    end   = output_text.rfind('}') + 1
    
    if start == -1 or end <= start:
        print(f"[Row {idx}] No JSON found. Output preview:\n{output_text[:200]}...\n")
        continue
    
    json_text = output_text[start:end]
    try:
        parsed = json.loads(json_text)
    except json.JSONDecodeError as e:
        print(f"[Row {idx}] JSON parsing error: {e}\nPayload:\n{json_text}\n")
        continue
    
    # Limit reasoning to first 50 words
    reasoning_full = parsed.get("reasoning", "")
    reasoning_words = reasoning_full.split()
    reasoning_snip  = " ".join(reasoning_words[:50])
    
    results.append({
        "ORIGINAL_ITEM_TEXT":   original_item_text,
        "SAMPLE_STORE":         sample_store,
        "PRODUCT_NUMBER":       product_number,
        "BRAND":                parsed.get("brand", "N/A"),
        "BRAND_SCORE":          parsed.get("brand_score", "N/A"),
        "CATEGORY":             parsed.get("category", "N/A"),
        "CATEGORY_SCORE":       parsed.get("category_score", "N/A"),
        "EXPANDED_DESCRIPTION": parsed.get("expanded_description", "N/A"),
        "REASONING_SNIPPET":    reasoning_snip
    })
    
    # Throttle requests to avoid rate limits
    time.sleep(0.2)

# Convert to DataFrame
results_df = pd.DataFrame(results)

# (Optional) Save to CSV
# results_df.to_csv('data/fetchGPT_results.csv', index=False)

# Show the first few rows
results_df.head()

,ORIGINAL_ITEM_TEXT,SAMPLE_STORE,PRODUCT_NUMBER,BRAND,BRAND_SCORE,CATEGORY,CATEGORY_SCORE,EXPANDED_DESCRIPTION,REASONING_SNIPPET
0,strawberries,COSTCO,27003,Unbranded,low,Produce,high,"Premium Strawberries, 2 lb clamshell","The product description 'strawberries' refers to fresh strawberries. The product number 27003 at Costco corresponds to 'Premium Strawberries, 2 lbs' ([costcobusinessdelivery.com](https://www.costcobusinessdelivery.com/premium-strawberries%2C-2-lbs.product.11576937.html?utm_source=openai)). No specific brand is associated with this item, indicating it is likely unbranded. Therefore, the brand confidence is low. The product falls under the 'Produce' category, with high confidence"
1,***kswtr40pk,COSTCO,782796,Kirkland Signature,high,Beverages,high,"Kirkland Signature Purified Drinking Water, 16.9 fl oz, 40-count","The original item text '***kswtr40pk' likely abbreviates 'Kirkland Signature Water 40 Pack.' The product number '782796' corresponds to Kirkland Signature Purified Drinking Water, 16.9 fl oz, 40-count at Costco. Therefore, the brand is Kirkland Signature, and the category is Beverages."
2,rotisserie,COSTCO,87745,Kirkland Signature,high,Meat & Seafood,high,Kirkland Signature Rotisserie Chicken,"The term 'rotisserie' refers to a cooking method involving meat cooked on a rotating spit, commonly associated with rotisserie chicken. Given the store context (Costco) and the product number 87745, this item is identified as the Kirkland Signature Rotisserie Chicken. The brand 'Kirkland Signature' is Costco's private label, and the"
3,ks cage free,COSTCO,637598,Kirkland Signature,high,Dairy,high,"Kirkland Signature Cage-Free Large Eggs, 24 Count","The abbreviation 'ks' corresponds to 'Kirkland Signature,' Costco's private label brand. 'Cage free' refers to the type of eggs. The product number 637598 at Costco matches 'Kirkland Signature Large Eggs, Cage Free, 2 Dozen,' confirming the product details."
4,18 ct eggs,COSTCO,1008,Kirkland Signature,high,Dairy,high,"Kirkland Signature Large Grade A Eggs, 18 Count","The product description '18 ct eggs' refers to an 18-count package of eggs. At Costco, item number 1008 corresponds to 'Large Eggs, Grade A, 18 ct' ([costcobusinessdelivery.com](https://www.costcobusinessdelivery.com/large-eggs%2C-grade-a%2C-18-ct.product.2001138745.html?utm_source=openai)). While the specific brand isn't explicitly stated, Costco frequently sells eggs under its Kirkland Signature brand. Given this context, it's reasonable to attribute"


In [ ]:
# Role and Objective
# Instructions
## Sub-categories for more detailed instructions
# Reasoning Steps
# Output Format
# Examples
## Example 1
# Context
# Final instructions and prompt to think step by step



In [83]:
# Merge the results_df with temp DataFrame
merged_df = pd.merge(
    temp, 
    results_df,
    on=['ORIGINAL_ITEM_TEXT', 'SAMPLE_STORE', 'PRODUCT_NUMBER'],
    how='left'
)

# Display the merged results
merged_df

# Optionally save to CSV
merged_df.to_csv('data/merged_results.csv', index=False)